In [1]:
"""https://arxiv.org/abs/1409.4842"""

'https://arxiv.org/abs/1409.4842'

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPool2D,\
BatchNormalization, Dense, Dropout, Activation, Flatten,\
Input, ZeroPadding2D, Add, GlobalAveragePooling2D, concatenate,\
AveragePooling2D

from tensorflow.keras import Model

import tensorflow as tf

In [3]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,\
                     filters_5x5_reduce,filters_5x5,filters_pool_proj):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3)
    
    return output

In [4]:
def get_model():
    input_layer = Input(shape=(224, 224, 3))

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu')(input_layer)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

    x = inception_module(x,64,96,128,16,32,32)
    x = inception_module(x,128,128,192,32,96,64)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    x = inception_module(x,192,96,208,16,48,64)

    y = AveragePooling2D((5, 5), strides=3)(x)
    y = Conv2D(128, (1, 1), padding='same', activation='relu')(y)
    y = Flatten()(y)
    y = Dense(1024, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(10, activation='softmax')(y)

    x = inception_module(x,160,112,224,24,64,64)
    x = inception_module(x,128,128,256,24,64,64)
    x = inception_module(x,112,144,288,32,64,64)

    z = AveragePooling2D((5, 5), strides=3)(x)
    z = Conv2D(128, (1, 1), padding='same', activation='relu')(z)
    z = Flatten()(z)
    z = Dense(1024, activation='relu')(z)
    z = Dropout(0.2)(z)
    z = Dense(10, activation='softmax')(z)

    x = inception_module(x,256,160,320,32,128,128)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    x = inception_module(x,256,160,320,32,128,128)
    x = inception_module(x,384,192,384,48,128,128)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(1000, activation='softmax')(x)
    
    model = Model(input_layer, [x, y,z])
    
    return model

In [5]:
get_model().summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d[0][0]              
______________________________________________________________________________________________